# XGCM Grid Index

In [1]:
%load_ext watermark

import xarray as xr
import xgcm
from xarray.testing import _assert_internal_invariants

%watermark -iv

xarray: 2023.1.1.dev28+ga9f4913af2
xgcm  : 0.8.1



In [2]:
import xgcm.test.datasets

ds, coords, metrics = xgcm.test.datasets.datasets_grid_metric("C")

In [3]:
import numpy as np
from xarray.core.indexes import Index, PandasIndex
from xarray.core.indexing import IndexSelResult


def grid_isel_indexers(grid: xgcm.Grid, indexers: dict):
    # TODO: make this a Grid.isel method
    # TODO: cache positions on the grid object.
    dim_to_pos = {}
    for key, axis in grid.axes.items():
        for pos, name in axis.coords.items():
            dim_to_pos.update({name: (key, pos)})

    new_indexers = {}
    new_indexers.update(indexers)
    for dim, key in indexers.items():
        axis, pos = dim_to_pos[dim]
        print(axis, pos)

        if np.isscalar(key):
            key = [key]
        key = np.asarray(key)

        for pos_, dim in grid.axes[axis].coords.items():
            if dim in new_indexers:
                continue
                
            if len(key) > 1:
                raise NotImplementedError
            if pos_ == pos:
                continue
            if pos_ == "right":
                new_indexers[dim] = np.insert(key, 0, key - 1)
            elif pos_ == "left":
                new_indexers[dim] = np.append(key, key + 1)
    print(new_indexers)
    return new_indexers


def get_grid_var_names(grid):
    import itertools

    return tuple(
        itertools.chain(*[tuple(axis.coords.values()) for axis in grid.axes.values()])
    ) + tuple(
        itertools.chain(
            *[tuple(v.name for v in metrics) for metrics in grid._metrics.values()]
        )
    )


class XgcmGridIndex(Index):
    # based off Benoit's RasterIndex in
    # https://hackmd.io/Zxw_zCa7Rbynx_iJu6Y3LA?view

    def __init__(self, grid, indexes):
        display("Creating new index... __init__")
        self.grid = grid

        # all variable names
        self.index_var_names = get_grid_var_names(grid)
        self._indexes = indexes

    # TODO: what goes in options?
    @classmethod
    def from_variables(cls, variables, options):
        grid = options.pop("grid")

        coord_names = []
        for _, axis in grid.axes.items():
            coord_names.extend(axis.coords.values())

        indexes = {
            key: PandasIndex.from_variables({key: variables[key]}, options=options)
            for key in coord_names
        }
        return cls(grid, indexes)

    # TODO: variables=None?
    # set_xindex tries to pass variables; this seems like a bug
    def create_variables(self, variables=None):
        return {name: self.grid._ds._variables[name] for name in self.index_var_names}

    # TODO: see notes about IndexSelResult
    #    The latter is a small class that stores positional indexers (indices)
    #    and that could also store new variables, new indexes,
    #    names of variables or indexes to drop,
    #    names of dimensions to rename, etc.
    def sel(self, labels, **kwargs):
        # sel needs to only handle keys in labels
        # since it delegates to isel.
        # we handle all entries in ._indexes there
        results = {}

        # convert provided label indexers to positional indexers
        for name, keys in labels.items():
            pdindex = self._indexes[name].index
            idxs = pdindex.get_indexer(keys)
            results[name] = idxs

        # bring in linked dimensions
        results = grid_isel_indexers(self.grid, results)
        return IndexSelResult(dim_indexers=results)

    def isel(self, indexers):
        indexers = grid_isel_indexers(self.grid, indexers)
        display(indexers)

        # TODO: check dim names in indexes
        results = {}
        for k, index in self._indexes.items():
            if k in indexers:
                results[k] = index.isel({k: indexers[k]})
            else:
                results[k] = index
        # display(results)
        subset_ds = self.grid._ds.isel(indexers)
        # display(subset_ds)
        new_grid = xgcm.Grid(
            subset_ds,
            coords=self.grid._saved_coords,
            metrics=self.grid._saved_metrics,
        )
        return type(self)(new_grid, results)

    def __repr__(self):
        string = f"XGCM/Index"
        return string


import itertools

ds, coords, metrics = xgcm.test.datasets.datasets_grid_metric("C")
grid = xgcm.Grid(ds, coords=coords, metrics=metrics)

# monkey patch attributes to make it easier to recreate a Grid
grid._saved_coords = coords
grid._saved_metrics = metrics

grid_var_names = tuple(
    itertools.chain(*[tuple(axis.coords.values()) for axis in grid.axes.values()])
) + tuple(
    itertools.chain(
        *[tuple(v.name for v in metrics) for metrics in grid._metrics.values()]
    )
)
dim_names = tuple(
    itertools.chain(*[tuple(axis.coords.values()) for axis in grid.axes.values()])
)
# spatial_dims = [dim for dim in ds.dims if dim != "time"]
# ds = ds.drop_indexes(spatial_dims)
newds = (
    # Set grid variables as coords
    ds.set_coords(grid_var_names)
    # Need to drop existing indexed dims; somewhat annoying
    .drop_indexes(dim_names).set_xindex(
        grid_var_names, index_cls=XgcmGridIndex, grid=grid
    )
)
newds

'Creating new index... __init__'

<xarray.Dataset>
Dimensions:     (xu: 4, yt: 5, time: 10, zt: 6, xt: 4, yu: 5, zw: 6)
Coordinates: (12/25)
  * time        (time) int64 0 1 2 3 4 5 6 7 8 9
  * xt          (xt) int64 0 1 2 3
  * xu          (xu) float64 0.5 1.5 2.5 3.5
  * yt          (yt) int64 0 1 2 3 4
  * yu          (yu) float64 0.5 1.5 2.5 3.5 4.5
  * zt          (zt) int64 0 1 2 3 4 5
    ...          ...
  * dz_w_e      (xu, yt, time, zw) float64 0.6532 13.25 11.57 ... 5.001 14.73
  * area_t      (xt, yt) float64 0.16 0.16 0.16 0.16 ... 0.16 0.16 0.16 0.16
  * area_n      (xt, yu) float64 0.42 0.42 0.42 0.42 ... 0.42 0.42 0.42 0.42
  * area_e      (xu, yt) float64 0.3 0.3 0.3 0.3 0.3 0.3 ... 0.3 0.3 0.3 0.3 0.3
  * area_ne     (xu, yu) float64 0.52 0.52 0.52 0.52 ... 0.52 0.52 0.52 0.52
  * volume_t    (xt, yt, time, zt) float64 -1.817 -4.549 ... -0.5453 -3.712
Data variables:
    u           (xu, yt, time, zt) float64 0.9733 0.9542 ... 0.1156 0.2594
    v           (xt, yu, time, zt) float64 0.1923 0.08497 ... 0.8069 0.6802
    wt          (xt, yt, time, zw) float64 0.4419 0.7739 ... 0.2743 0.9626
    tracer      (xt, yt, time, zt) float64 0.3421 0.8334 0.424 ... 0.37 0.0206
    timeseries  (time) float64 0.5413 0.2879 0.7025 ... 0.3741 0.921 0.4959
Indexes: (12/24)
    xt          XgcmGridIndex
    xu          XgcmGridIndex
    yt          XgcmGridIndex
    yu          XgcmGridIndex
    zt          XgcmGridIndex
    zw          XgcmGridIndex
    ...          ...
    dz_w_e      XgcmGridIndex
    area_t      XgcmGridIndex
    area_n      XgcmGridIndex
    area_e      XgcmGridIndex
    area_ne     XgcmGridIndex
    volume_t    XgcmGridIndex

## Index gets propagated

Index is propagated with mean

In [4]:
newds.mean("time")

<xarray.Dataset>
Dimensions:     (xu: 4, yt: 5, zt: 6, xt: 4, yu: 5, zw: 6)
Coordinates: (12/18)
  * xt          (xt) int64 0 1 2 3
  * xu          (xu) float64 0.5 1.5 2.5 3.5
  * yt          (yt) int64 0 1 2 3 4
  * yu          (yu) float64 0.5 1.5 2.5 3.5 4.5
  * zt          (zt) int64 0 1 2 3 4 5
  * zw          (zw) float64 0.5 1.5 2.5 3.5 4.5 5.5
    ...          ...
  * dy_e        (xu, yt) float64 2.3 2.3 2.3 2.3 2.3 2.3 ... 2.3 2.3 2.3 2.3 2.3
  * dy_ne       (xu, yu) float64 2.1 2.1 2.1 2.1 2.1 2.1 ... 2.1 2.1 2.1 2.1 2.1
  * area_t      (xt, yt) float64 0.16 0.16 0.16 0.16 ... 0.16 0.16 0.16 0.16
  * area_n      (xt, yu) float64 0.42 0.42 0.42 0.42 ... 0.42 0.42 0.42 0.42
  * area_e      (xu, yt) float64 0.3 0.3 0.3 0.3 0.3 0.3 ... 0.3 0.3 0.3 0.3 0.3
  * area_ne     (xu, yu) float64 0.52 0.52 0.52 0.52 ... 0.52 0.52 0.52 0.52
Data variables:
    u           (xu, yt, zt) float64 0.471 0.6668 0.5021 ... 0.589 0.5689 0.5481
    v           (xt, yu, zt) float64 0.5041 0.4941 0.3631 ... 0.6758 0.5681
    wt          (xt, yt, zw) float64 0.469 0.5299 0.5875 ... 0.4725 0.6713
    tracer      (xt, yt, zt) float64 0.613 0.5962 0.4378 ... 0.6587 0.3525
    timeseries  float64 0.6407
Indexes: (12/18)
    xt          XgcmGridIndex
    xu          XgcmGridIndex
    yt          XgcmGridIndex
    yu          XgcmGridIndex
    zt          XgcmGridIndex
    zw          XgcmGridIndex
    ...          ...
    dy_e        XgcmGridIndex
    dy_ne       XgcmGridIndex
    area_t      XgcmGridIndex
    area_n      XgcmGridIndex
    area_e      XgcmGridIndex
    area_ne     XgcmGridIndex

## Subset with .sel

In [5]:
newds.sel(xt=[2])

X center
{'xt': array([2]), 'xu': array([1, 2])}
X center
X right
{'xt': array([2]), 'xu': array([1, 2])}


{'xt': array([2]), 'xu': array([1, 2])}

'Creating new index... __init__'

<xarray.Dataset>
Dimensions:     (time: 10, xu: 2, yt: 5, zt: 6, xt: 1, yu: 5, zw: 6)
Coordinates: (12/25)
  * time        (time) int64 0 1 2 3 4 5 6 7 8 9
  * xt          (xt) int64 2
  * xu          (xu) float64 1.5 2.5
  * yt          (yt) int64 0 1 2 3 4
  * yu          (yu) float64 0.5 1.5 2.5 3.5 4.5
  * zt          (zt) int64 0 1 2 3 4 5
    ...          ...
  * dz_w_e      (xu, yt, time, zw) float64 2.655 8.975 16.44 ... 13.56 8.137
  * area_t      (xt, yt) float64 0.16 0.16 0.16 0.16 0.16
  * area_n      (xt, yu) float64 0.42 0.42 0.42 0.42 0.42
  * area_e      (xu, yt) float64 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3 0.3
  * area_ne     (xu, yu) float64 0.52 0.52 0.52 0.52 ... 0.52 0.52 0.52 0.52
  * volume_t    (xt, yt, time, zt) float64 -1.93 -1.538 -1.896 ... -0.775 -3.503
Data variables:
    u           (xu, yt, time, zt) float64 0.1438 0.529 ... 0.6756 0.1779
    v           (xt, yu, time, zt) float64 0.3769 0.3952 ... 0.0003817 0.1645
    wt          (xt, yt, time, zw) float64 0.5052 0.8866 ... 0.5931 0.8931
    tracer      (xt, yt, time, zt) float64 0.2948 0.8328 ... 0.2549 0.2665
    timeseries  (time) float64 0.5413 0.2879 0.7025 ... 0.3741 0.921 0.4959
Indexes: (12/24)
    xt          XgcmGridIndex
    xu          XgcmGridIndex
    yt          XgcmGridIndex
    yu          XgcmGridIndex
    zt          XgcmGridIndex
    zw          XgcmGridIndex
    ...          ...
    dz_w_e      XgcmGridIndex
    area_t      XgcmGridIndex
    area_n      XgcmGridIndex
    area_e      XgcmGridIndex
    area_ne     XgcmGridIndex
    volume_t    XgcmGridIndex

Not sure why this is failing. Seems like I need to make a bunch of variables IndexVariables?

In [6]:
_assert_internal_invariants(newds.sel(xt=[2]), check_default_indexes=False)

X center
{'xt': array([2]), 'xu': array([1, 2])}
X center
X right
{'xt': array([2]), 'xu': array([1, 2])}


{'xt': array([2]), 'xu': array([1, 2])}

'Creating new index... __init__'

AssertionError: ({'dx_n', 'area_t', 'dz_w_ne', 'xt', 'area_ne', 'yu', 'dx_t', 'dy_n', 'xu', 'dy_ne', 'time', 'dz_t', 'dx_e', 'zt', 'volume_t', 'yt', 'area_n', 'dz_w_n', 'dz_w', 'dz_w_e', 'dx_ne', 'dy_e', 'zw', 'area_e', 'dy_t'}, {'zt', 'xt', 'yt', 'yu', 'xu', 'time', 'zw'})

## Subset with .isel

This isn't working, `indexers={"xt": [2]}` needs to be updated before subsetting variables but we have no mechanism to allow that. We'd need `IndexISelResult` for example? Or convert from positional to label, then back to positional is quite silly.

In [7]:
_assert_internal_invariants(newds.mean("time").isel(xt=[2]), False)

X center
{'xt': [2], 'xu': array([1, 2])}


{'xt': [2], 'xu': array([1, 2])}

'Creating new index... __init__'

AssertionError: ({'xu': 2, 'yt': 5, 'zt': 6, 'xt': 1, 'yu': 5, 'zw': 6}, {'u': Frozen({'xu': 4, 'yt': 5, 'zt': 6}), 'v': Frozen({'xt': 1, 'yu': 5, 'zt': 6}), 'wt': Frozen({'xt': 1, 'yt': 5, 'zw': 6}), 'tracer': Frozen({'xt': 1, 'yt': 5, 'zt': 6}), 'timeseries': Frozen({}), 'xt': Frozen({'xt': 1}), 'xu': Frozen({'xu': 2}), 'yt': Frozen({'yt': 5}), 'yu': Frozen({'yu': 5}), 'zt': Frozen({'zt': 6}), 'zw': Frozen({'zw': 6}), 'dx_t': Frozen({'xt': 1, 'yt': 5}), 'dx_n': Frozen({'xt': 1, 'yu': 5}), 'dx_e': Frozen({'xu': 2, 'yt': 5}), 'dx_ne': Frozen({'xu': 2, 'yu': 5}), 'dy_t': Frozen({'xt': 1, 'yt': 5}), 'dy_n': Frozen({'xt': 1, 'yu': 5}), 'dy_e': Frozen({'xu': 2, 'yt': 5}), 'dy_ne': Frozen({'xu': 2, 'yu': 5}), 'area_t': Frozen({'xt': 1, 'yt': 5}), 'area_n': Frozen({'xt': 1, 'yu': 5}), 'area_e': Frozen({'xu': 2, 'yt': 5}), 'area_ne': Frozen({'xu': 2, 'yu': 5})})